In [ ]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R
from itertools import product

In [ ]:
(
    reference_x,
    reference_y,
) = sympy.symbols("x y")

f_interpolation = sympy.Matrix(sympy.symbols("f_0:3"))
s = sympy.symbols("s")

In [ ]:
v0 = {reference_x: 0, reference_y: 0}
v1 = {reference_x: 1, reference_y: 0}
v2 = {reference_x: 0, reference_y: 1}

# Get basis functions

In [ ]:
monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
]
)

In [ ]:
Vander = sympy.Matrix(np.zeros((3, 3)))

for p_e_idx, basis in enumerate(monomial_basis):
    
    Vander[p_e_idx, 0] = basis.subs(v0)
    Vander[p_e_idx, 1] = basis.subs(v1)
    Vander[p_e_idx, 2] = basis.subs(v2)

In [ ]:
lagrange_p1_basis = Vander.inv()@monomial_basis

In [ ]:
(    
    physical_x_0_first,
    physical_y_0_first,
    physical_x_1_first,
    physical_y_1_first,
    physical_x_2_first,
    physical_y_2_first,
    physical_x_0_second,
    physical_y_0_second,
    physical_x_1_second,
    physical_y_1_second,
    physical_x_2_second,
    physical_y_2_second,
) = sympy.symbols(r"x_0^{first} y_0^{first} x_1^{first} y_1^{first} x_2^{first} y_2^{first} x_0^{second} y_0^{second} x_1^{second} y_1^{second} x_2^{second} y_2^{second}")


In [ ]:
rotation_matrix = sympy.Matrix(
    [
        [+R(0), +R(1)],
        [-R(1), +R(0)]
    ]
)

l0_p = ((physical_x_2_first - physical_x_1_first) ** 2 + (physical_y_2_first - physical_y_1_first) ** 2) ** R(1, 2)
l1_p = ((physical_x_2_first - physical_x_0_first) ** 2 + (physical_y_2_first - physical_y_0_first) ** 2) ** R(1, 2)
l2_p = ((physical_x_1_first - physical_x_0_first) ** 2 + (physical_y_1_first - physical_y_0_first) ** 2) ** R(1, 2)

t0_p = sympy.Matrix([physical_x_2_first - physical_x_1_first, physical_y_2_first - physical_y_1_first]) / l0_p
t1_p = sympy.Matrix([physical_x_2_first - physical_x_0_first, physical_y_2_first - physical_y_0_first]) / l1_p
t2_p = sympy.Matrix([physical_x_1_first - physical_x_0_first, physical_y_1_first - physical_y_0_first]) / l2_p

n0_p = rotation_matrix @ t0_p
n1_p = rotation_matrix @ t1_p
n2_p = rotation_matrix @ t2_p

l0_n = ((physical_x_2_second - physical_x_1_second) ** 2 + (physical_y_2_second - physical_y_1_second) ** 2) ** R(1, 2)
l1_n = ((physical_x_2_second - physical_x_0_second) ** 2 + (physical_y_2_second - physical_y_0_second) ** 2) ** R(1, 2)
l2_n = ((physical_x_1_second - physical_x_0_second) ** 2 + (physical_y_1_second - physical_y_0_second) ** 2) ** R(1, 2)

t0_n = sympy.Matrix([physical_x_2_second - physical_x_1_second, physical_y_2_second - physical_y_1_second]) / l0_n
t1_n = sympy.Matrix([physical_x_2_second - physical_x_0_second, physical_y_2_second - physical_y_0_second]) / l1_n
t2_n = sympy.Matrix([physical_x_1_second - physical_x_0_second, physical_y_1_second - physical_y_0_second]) / l2_n

n0_n = rotation_matrix @ t0_n
n1_n = rotation_matrix @ t1_n
n2_n = rotation_matrix @ t2_n

In [ ]:
S = R(1, 2)*(l0_p + l1_p + l2_p)
r = sympy.sqrt((S - l0_p) * (S - l1_p) * (S - l2_p))

# Calculate Jacobians

In [ ]:
mapping_basis = Vander.inv() @ monomial_basis

global_x_first = (
      mapping_basis[0] * physical_x_0_first
    + mapping_basis[1] * physical_x_1_first
    + mapping_basis[2] * physical_x_2_first
)
global_y_first = (
      mapping_basis[0] * physical_y_0_first
    + mapping_basis[1] * physical_y_1_first
    + mapping_basis[2] * physical_y_2_first
)
mapping_function_first = sympy.Matrix([global_x_first, global_y_first])
J_cofactor_T_first = mapping_function_first.jacobian([reference_x, reference_y]).cofactor_matrix().T
J_p = J_cofactor_T_first / J_cofactor_T_first.det()
J_inv_first = J_p.inv()


global_x_second = (
      mapping_basis[0] * physical_x_0_second
    + mapping_basis[1] * physical_x_1_second
    + mapping_basis[2] * physical_x_2_second
)
global_y_second = (
      mapping_basis[0] * physical_y_0_second
    + mapping_basis[1] * physical_y_1_second
    + mapping_basis[2] * physical_y_2_second
)
mapping_function_second = sympy.Matrix([global_x_second, global_y_second])
J_cofactor_T_second = mapping_function_second.jacobian([reference_x, reference_y]).cofactor_matrix().T
J_n = J_cofactor_T_second / J_cofactor_T_second.det()
J_inv_second = J_n.inv()

# Calculate symbolic integrals

In [ ]:
N = len(lagrange_p1_basis)

consistency =   np.zeros((2, 2, 3, 3, N, N), dtype=object)
symmetricity =  np.zeros((2, 2, 3, 3, N, N), dtype=object)
penalty =       np.zeros((2, 2, 3, 3, N, N), dtype=object)

consistency_boundary =  np.zeros((3, N, N), dtype=object)
symmetricity_boundary = np.zeros((3, N, N), dtype=object)
penalty_boundary =      np.zeros((3, N, N), dtype=object)

bc_dirichlet = np.zeros((3, N, N), dtype=object)
bc_neumann = np.zeros((3, N, N), dtype=object)

e0 = {reference_x: 1-s, reference_y: s}
e1 = {reference_x: 0, reference_y: s}
e2 = {reference_x: s, reference_y: 0}

integration_edges = [e0, e1, e2]
lengths_p = [l0_p, l1_p, l2_p]
lengths_n = [l0_n, l1_n, l2_n]

normals_p = [n0_p, n1_p, n2_p]
normals_n = [n0_n, n1_n, n2_n]

gamma = sympy.symbols('\\gamma')


# Element integrals
for trial_idx, test_idx, p_e_idx, n_e_idx in tqdm(product(range(N), range(N), range(3), range(3)), total=3*3*N*N):
        
    u_p = lagrange_p1_basis[trial_idx].subs(integration_edges[p_e_idx])
    v_p = lagrange_p1_basis[test_idx].subs(integration_edges[p_e_idx])
    u_n = lagrange_p1_basis[trial_idx].subs(integration_edges[n_e_idx])
    v_n = lagrange_p1_basis[test_idx].subs(integration_edges[n_e_idx])
    
    # Gradients
    u_p_grad_ = ((sympy.Matrix([lagrange_p1_basis[trial_idx]]).jacobian([reference_x, reference_y])@J_p)@normals_p[p_e_idx]).subs(integration_edges[p_e_idx])
    u_n_grad_ = ((sympy.Matrix([lagrange_p1_basis[trial_idx]]).jacobian([reference_x, reference_y])@J_n)@normals_n[n_e_idx]).subs(integration_edges[n_e_idx])
    v_p_grad_ = ((sympy.Matrix([lagrange_p1_basis[test_idx]]).jacobian([reference_x, reference_y])@J_p)@normals_p[p_e_idx]).subs(integration_edges[p_e_idx])
    v_n_grad_ = ((sympy.Matrix([lagrange_p1_basis[test_idx]]).jacobian([reference_x, reference_y])@J_n)@normals_n[n_e_idx]).subs(integration_edges[n_e_idx])

    # Consistency
    consistency[0, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = +R(1, 2) * (sympy.integrate(u_p_grad_*v_p, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    consistency[0, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = -R(1, 2) * (sympy.integrate(u_p_grad_*v_n, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    consistency[1, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = +R(1, 2) * (sympy.integrate(u_n_grad_*v_p, (s, 0, 1)) * lengths_n[n_e_idx])[0, 0]
    consistency[1, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = -R(1, 2) * (sympy.integrate(u_n_grad_*v_n, (s, 0, 1)) * lengths_n[n_e_idx])[0, 0]

    # Symmetricity
    symmetricity[0, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = +R(1, 2) * (sympy.integrate(u_p*v_p_grad_, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    symmetricity[0, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = +R(1, 2) * (sympy.integrate(u_p*v_n_grad_, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    symmetricity[1, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = -R(1, 2) * (sympy.integrate(u_n*v_p_grad_, (s, 0, 1)) * lengths_n[n_e_idx])[0, 0]
    symmetricity[1, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = -R(1, 2) * (sympy.integrate(u_n*v_n_grad_, (s, 0, 1)) * lengths_n[n_e_idx])[0, 0]

    # Penalty    
    penalty[0, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = + sympy.integrate(u_p*v_p, (s, 0, 1)) * lengths_p[p_e_idx] * gamma / r
    penalty[0, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = - sympy.integrate(u_p*v_n, (s, 0, 1)) * lengths_p[p_e_idx] * gamma / r
    penalty[1, 0][p_e_idx, n_e_idx][trial_idx, test_idx] = - sympy.integrate(u_n*v_p, (s, 0, 1)) * lengths_n[n_e_idx] * gamma / r
    penalty[1, 1][p_e_idx, n_e_idx][trial_idx, test_idx] = + sympy.integrate(u_n*v_n, (s, 0, 1)) * lengths_n[n_e_idx] * gamma / r


# Facet integrals
for trial_idx, test_idx, p_e_idx in tqdm(product(range(N), range(N), range(3)), total=3*N*N):
        
    u_p = lagrange_p1_basis[trial_idx].subs(integration_edges[p_e_idx])
    v_p = lagrange_p1_basis[test_idx].subs(integration_edges[p_e_idx])
    u_n = lagrange_p1_basis[trial_idx].subs(integration_edges[n_e_idx])
    v_n = lagrange_p1_basis[test_idx].subs(integration_edges[n_e_idx])
    
    # Gradients
    u_p_grad_ = ((sympy.Matrix([lagrange_p1_basis[trial_idx]]).jacobian([reference_x, reference_y])@J_p)@normals_p[p_e_idx]).subs(integration_edges[p_e_idx])
    u_n_grad_ = ((sympy.Matrix([lagrange_p1_basis[trial_idx]]).jacobian([reference_x, reference_y])@J_n)@normals_n[n_e_idx]).subs(integration_edges[n_e_idx])
    v_p_grad_ = ((sympy.Matrix([lagrange_p1_basis[test_idx]]).jacobian([reference_x, reference_y])@J_p)@normals_p[p_e_idx]).subs(integration_edges[p_e_idx])
    v_n_grad_ = ((sympy.Matrix([lagrange_p1_basis[test_idx]]).jacobian([reference_x, reference_y])@J_n)@normals_n[n_e_idx]).subs(integration_edges[n_e_idx])

    # SIP
    consistency_boundary[p_e_idx][trial_idx, test_idx] =     (sympy.integrate(u_p_grad_*v_p, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    symmetricity_boundary[p_e_idx][trial_idx, test_idx] =    (sympy.integrate(u_p*v_p_grad_, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0]
    penalty_boundary[p_e_idx][trial_idx, test_idx] =         sympy.integrate(u_p*v_p, (s, 0, 1)) * lengths_p[p_e_idx] * gamma / r

    # Dirichlet
    g = lagrange_p1_basis[trial_idx].subs(integration_edges[p_e_idx])
    h = lagrange_p1_basis[trial_idx].subs(integration_edges[p_e_idx])

    # Neumann
    bc_dirichlet[p_e_idx][trial_idx, test_idx] = (-sympy.integrate(g*v_p_grad_, (s, 0, 1)) * lengths_p[p_e_idx])[0, 0] + sympy.integrate(g*v_p, (s, 0, 1)) * lengths_p[p_e_idx] * gamma / r
    bc_neumann[p_e_idx][trial_idx, test_idx] = sympy.integrate(h*v_p, (s, 0, 1)) * lengths_p[p_e_idx]
                        
consistency = sympy.Array(consistency)
symmetricity = sympy.Array(symmetricity)
penalty = sympy.Array(penalty)

consistency_boundary = sympy.Array(consistency_boundary)
symmetricity_boundary = sympy.Array(symmetricity_boundary)
penalty_boundary = sympy.Array(penalty_boundary)

bc_dirichlet = sympy.Array(bc_dirichlet)
bc_neumann = sympy.Array(bc_neumann)

# Save the functions

In [ ]:
lambdify_symbols = (
    reference_x,
    reference_y,
    physical_x_0_first,
    physical_y_0_first,
    physical_x_1_first,
    physical_y_1_first,
    physical_x_2_first,
    physical_y_2_first,
)

In [ ]:
interpolation_function = (lagrange_p1_basis.T @ f_interpolation)[0, 0]

interpolation_function_lambdified = sympy.lambdify(
    [reference_x, reference_y, *f_interpolation],
    interpolation_function,
    cse=True,
)

mapping_function_lambdified = sympy.lambdify(
    lambdify_symbols, mapping_function_first[:, 0], cse=True
)

In [ ]:
dill.dump(interpolation_function_lambdified, open("../../calculations/lagrange_linear_interpolation_x", "wb"))
dill.dump(mapping_function_lambdified, open("../../calculations/lagrange_linear_mapping_x", "wb"))

In [ ]:
lambdify_symbols_element = (    
    physical_x_0_first,
    physical_y_0_first,
    physical_x_1_first,
    physical_y_1_first,
    physical_x_2_first,
    physical_y_2_first,
    physical_x_0_second,
    physical_y_0_second,
    physical_x_1_second,
    physical_y_1_second,
    physical_x_2_second,
    physical_y_2_second,
)

In [ ]:
consistency_lambdified = sympy.lambdify(lambdify_symbols_element, consistency, cse=True)
symmetricity_lambdified = sympy.lambdify(lambdify_symbols_element, symmetricity, cse=True)
penalty_lambdified = sympy.lambdify(lambdify_symbols_element + (gamma, ), penalty, cse=True)

In [ ]:
dill.dump(consistency_lambdified, open("../../calculations/lagrange_linear_poisson_dg_consistency", "wb"))
dill.dump(symmetricity_lambdified, open("../../calculations/lagrange_linear_poisson_dg_symmetricity", "wb"))
dill.dump(penalty_lambdified, open("../../calculations/lagrange_linear_poisson_dg_penalty", "wb"))

In [ ]:
lambdify_symbols_facet = (    
    physical_x_0_first,
    physical_y_0_first,
    physical_x_1_first,
    physical_y_1_first,
    physical_x_2_first,
    physical_y_2_first,
)

In [ ]:
consistency_boundary_lambdified = sympy.lambdify(lambdify_symbols_facet, consistency_boundary, cse=True)
symmetricity_boundary_lambdified = sympy.lambdify(lambdify_symbols_facet, symmetricity_boundary, cse=True)
penalty_boundary_lambdified = sympy.lambdify(lambdify_symbols_facet + (gamma,), penalty_boundary, cse=True)

In [ ]:
dill.dump(consistency_boundary_lambdified, open("../../calculations/lagrange_linear_poisson_dg_consistency_boundary", "wb"))
dill.dump(symmetricity_boundary_lambdified, open("../../calculations/lagrange_linear_poisson_dg_symmetricity_boundary", "wb"))
dill.dump(penalty_boundary_lambdified, open("../../calculations/lagrange_linear_poisson_dg_penalty_boundary", "wb"))

In [ ]:
bc_dirichlet_lambdified = sympy.lambdify(lambdify_symbols_facet + (gamma, ), bc_dirichlet, cse=True)
bc_neumann_lambdified = sympy.lambdify(lambdify_symbols_facet, bc_neumann, cse=True)

In [ ]:
dill.dump(bc_dirichlet_lambdified, open("../../calculations/lagrange_linear_poisson_dg_bc_dirichlet", "wb"))
dill.dump(bc_neumann_lambdified, open("../../calculations/lagrange_linear_poisson_dg_bc_neumann", "wb"))